# A\) Etapas do Pré - Processamento
## 2) Fazer a leitura do xls que foi criado na etapa anterior e retirar NAN'S
## 3\) Fazer a leitura do xls anterior e retirar texto inicial e final que não será objeto do estudo

> 
>Utilizaremos programação Python com biblioteca PANDAS e Expressão Regular
> 

In [ ]:
import glob
import pandas as pd
import re
from pypdf import PdfReader

In [ ]:
diretorio_planilhas = "C:\\DataSets\\Projeto_Leitura_Documentos\\planilhas"

In [ ]:
def substituir(texto_atual,texto_novo,regex):
    return regex.sub(texto_novo,texto_atual)

In [ ]:
import regex

def retorna_data(texto):
    return regex.search("rio branco  ([0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9])",texto).group(1)


# Continuando a partir do acordao 01
# Apagando coluna 'Unnamed: 0'

In [ ]:
transitorio = pd.read_csv(f"{diretorio_planilhas}\\transitorio.csv",index_col=[0])
passo01 = pd.read_csv(f"{diretorio_planilhas}\\acordaos01.csv",index_col=[0])

In [ ]:
# atualiza
passo01 = passo01
passo01.head()

# Leitura registro a registro

In [ ]:
def retorna_conteudo(lista,indice):
  return lista.conteudo[indice]

In [ ]:
def quantidade_ocorrencias(tabela,expressao_regular_a_procurar):
    resultado = []
    for i in tabela.index:
        previa = re.findall(expressao_regular_a_procurar,tabela.conteudo[i])
        if previa != []:
            resultado = resultado + previa
    return f"Encontrado {len(resultado)} vezes"

# Fazendo pesquisa nos registros:

In [ ]:
quantidade_ocorrencias(passo01," A C Ó R D Ã O ")

In [ ]:
def substituir(texto_original,novo_texto,regex):
    return regex.sub(novo_texto,texto_original)

# Fazendo pré-compilação do regex (expressão regular) para ganhar desempenho

In [ ]:
regex_stado_do_acre = re.compile(r"^stado do acre\.")
regex_acordao = re.compile(r"a c ó r d ã o")
regex_pontos = re.compile(r"\.")
regex_tracos = re.compile(r"(-|-){1,}")
regex_acordam = re.compile(r" a c o r d a m ")
regex_espacos_em_branco_em_duplicidade = re.compile(r"( {2,})")
regex_tribunal_secretaria = re.compile(r"tribunal de contas do estado do acre(.|\n)secretaria das sessões")
regex_missao = re.compile(r"(missão)(:|)(.*)(social)")
regex_endereco = re.compile(r"(av ceará)(.|\n)*(govbr)")
regex_sala_das_sessoes = re.compile(r"sala das sessões do tribunal de contas do estado do acre")
regex_dia_sessao = re.compile(r"(rio branco )(.|)( acre)(,|)( [0-9][0-9] de | [0-9] de )(janeiro|fevereiro|março|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)( de )([0-9][0-9][0-9][0-9])(\.|)")
regex_virgula = re.compile(r"( ,|,)")

#Padronizando as datas
regex_janeiro = re.compile(r"( de janeiro de )")
regex_fevereiro = re.compile(r"( de fevereiro de )")
regex_marco = re.compile(r"( de março de )")
regex_abril = re.compile(r"( de abril de )")
regex_maio = re.compile(r"( de maio de )")
regex_junho = re.compile(r"( de junho de )")
regex_julho = re.compile(r"( de julho de )")
regex_agosto = re.compile(r"( de agosto de )")
regex_setembro = re.compile(r"( de setembro de )")
regex_outubro = re.compile(r"( de outubro de )")
regex_novembro = re.compile(r"( de novembro de )")
regex_dezembro = re.compile(r"( de dezembro de )")
regex_data = re.compile(r"(rio branco)( |, )([0-9][0-9])([0-9][0-9])([0-9][0-9][0-9][0-9])")
regex_ultimas_linhas = re.compile(r"(<fim?>)(.|\n){1,}")



#Retirando as pontuações
regex_pontuacao = re.compile(r"[^\w\s]")
regex_apagar_01 = re.compile(r"(tribunal de contas do estado do acre |secretaria das sessões | natureza do feito| feito | n )")
regex_numeros = re.compile(r"[0-9]")

regex_numero_acordao = re.compile(r"(acórdão n |acórdão ni |acórdão n. |acórdão n º )")
regex_quebra_de_linha = re.compile(r"(\n){1,}")
regex_complemento_acordao = re.compile(r"(plen)(á|a)(riotceac)")
regex_tceac_ss = re.compile(r"(tribunal de contas do estado do acre)(.*)(secretaria das sessões)")
regex_acordao = re.compile(r"(a c ó r d ã o)")
regex_ate_acordao = re.compile('^(.*)(br|)( |)(acórdão nº{1})')
regex_espacos_em_branco_em_duplicidade = re.compile(r"( {2,})")
regex_acordao_acordao = re.compile(r"(acórdão acórdão)")
regex_acordao_complemento = re.compile(r"(plenáriotceac|plenariotceac)")
regex_especiais = re.compile(r"[ºª]")
regex_nao_visuais = re.compile(r"(\r|\t){1,}")

regex_apagar_02 = re.compile("(acórdão |processo |tceacre |tceac |tce |tribunal de contas do estado do acre |secretaria das sessões |anexos |assunto )")
regex_uma_a_duas_letras = re.compile(" [a-z]{1,2} ")

In [ ]:
# ATENÇÃO: Todas substituição com Expressão Regular serão executadas nesta etapa Grupo_NLP_02 

In [ ]:
def fazer_primeiras_substituicoes(tabela):
    print('Aguarde', end = "")
    for i in tabela.index:
        #padronizando em letras minúsculas
        antes = tabela.conteudo[i].lower()
        
        depois = substituir(antes," ",regex_stado_do_acre)
        depois = substituir(depois," ",regex_complemento_acordao)
        depois = substituir(depois,"",regex_especiais)
        depois = substituir(depois," ",regex_nao_visuais)
        depois = substituir(depois," acórdão ",regex_acordao)
        depois = substituir(depois,"",regex_pontos)
        depois = substituir(depois," ",regex_tracos)
        depois = substituir(depois," acordam ",regex_acordam)
        depois = substituir(depois," ",regex_tribunal_secretaria)
        depois = substituir(depois," ",regex_missao)
        depois = substituir(depois," ",regex_endereco)
        depois = substituir(depois," ",regex_sala_das_sessoes)
        depois = substituir(depois,r"\1\4\5\6\7\8",regex_dia_sessao)
        
        depois = substituir(depois," ",regex_virgula)
        depois = substituir(depois,"-01-",regex_janeiro)
        depois = substituir(depois,"-02-",regex_fevereiro)
        depois = substituir(depois,"-03-",regex_marco)
        depois = substituir(depois,"-04-",regex_abril)
        depois = substituir(depois,"-05-",regex_maio)
        depois = substituir(depois,"-06-",regex_junho)
        depois = substituir(depois,"-07-",regex_julho)
        depois = substituir(depois,"-08-",regex_agosto)
        depois = substituir(depois,"-09-",regex_setembro)
        depois = substituir(depois,"-10-",regex_outubro)
        depois = substituir(depois,"-11-",regex_novembro)
        depois = substituir(depois,"-12-",regex_dezembro)
        depois = substituir(depois,r"\2\3\3",regex_data)
        
        depois = substituir(depois," ",regex_ultimas_linhas)
        depois = substituir(depois,"acórdão ",regex_numero_acordao)
        depois = substituir(depois,"",regex_pontuacao)
        depois = substituir(depois," ",regex_quebra_de_linha)
        
        depois = substituir(depois," ",regex_apagar_01)
        depois = substituir(depois," ",regex_espacos_em_branco_em_duplicidade)

        tabela.loc[i,'conteudo_atualizado'] = depois
        print('.', end = "")

    print('\nTexto processado com sucesso !')

In [ ]:
#Substituições antes de pegar o acórdão e a data
fazer_primeiras_substituicoes(passo01)
passo01.head() 

In [ ]:
# atualizando acordao02
transitorio['indice'] = passo01['indice']
transitorio['acordao'] = passo01['indice']
transitorio['dia'] = passo01['indice']
transitorio['conteudo'] = passo01['conteudo_atualizado']
transitorio.to_csv(f"{diretorio_planilhas}\\acordaos02.csv")

In [ ]:
transitorio.head

In [ ]:
for i in transitorio.index:
    #padronizando em letras minúsculas
    texto = transitorio.conteudo[i]
    dia = re.findall("(rio branco)( |, )([0-9][0-9])([0-9][0-9])([0-9][0-9][0-9][0-9])",texto)
    if len(dia) == 1:
        transitorio.dia[i] = dia[0][2]+'/'+dia[0][3]+'/'+dia[0][4]
    else:
        transitorio.dia[i] = "00/00/0000"

In [ ]:
transitorio.dia[129]

In [ ]:
for i in transitorio.index:
    #padronizando em letras minúsculas
    texto = transitorio.conteudo[i]
    acordao = re.findall("(acórdão)( n. | n | )([0-9]{1,})( )",texto)
    if len(acordao) >= 1:
        transitorio.acordao[i] = acordao[0][2]
    else:
        transitorio.acordao[i] = "000000"

In [ ]:
transitorio.head

In [ ]:
def fazer_substituicoes_finais(tabela):
    print('Aguarde', end = "")
    for i in tabela.index:

        #padronizando em letras minúsculas
        antes = tabela.conteudo[i].lower()
        
        depois = substituir(antes," ",regex_numeros)
        depois = substituir(depois," ",regex_apagar_02)
        depois = substituir(depois," ",regex_uma_a_duas_letras)
        depois = substituir(depois," ",regex_espacos_em_branco_em_duplicidade)
        depois = substituir(depois," ",regex_nao_visuais)

        tabela.loc[i,'conteudo_atualizado'] = depois
        print('.', end = "")

    print('\nTexto processado com sucesso !')

In [ ]:
#Substituições após pegar o acórdão e a data
fazer_substituicoes_finais(transitorio)
transitorio.head()

In [ ]:
# atualizando acordao02
transitorio['conteudo'] = transitorio['conteudo_atualizado']
transitorio = transitorio.drop("conteudo_atualizado", axis = 1)
transitorio.to_csv(f"{diretorio_planilhas}\\acordaos02.csv")
transitorio.head()

In [ ]:
transitorio.to_csv(f"{diretorio_planilhas}\\acordaos03.csv")

In [ ]:
transitorio.dia[140]

In [ ]:
transitorio.acordao[140] 

# Continuar no notebook Grupo_NLP_03.ipynb